Basic Algorithm: 
    Load data set from Kaggle
    Preprocess data into single data set -> HOW THOUGH
    Create test and train data sets from original set
    Train basic regressive model
    Create probablities for each team
    
Additions to this:
    Use of either bagging ensemble or random forest to create better model
    Addition of other data sets (referees or home/away)
    Change weights on data sets
    Preprocess irrelevant data

In [6]:
import pandas as pd;
import numpy as np;

#massey = pd.read_csv('massey_ordinals.csv')
#rsCompact = pd.read_csv('RegularSeasonCompactResults.csv')
rsDetailed = pd.read_csv('RegularSeasonDetailedResults.csv')
#Seasons = pd.read_csv('Seasons.csv')
#Teams = pd.read_csv('Teams.csv')
#tCompact = pd.read_csv('TourneyCompactResults.csv')
#tDetailed = pd.read_csv('TourneyDetailedResults.csv')
#tSlots = pd.read_csv('TourneySlots.csv')
tSeeds = pd.read_csv('TourneySeeds.csv')


In [7]:
#rsDetailed.head()
#tSlots.head()

Seed16 = tSeeds.loc[tSeeds['Season'] == 2016]
Teams16 = Seed16['Team']
rsData16 = rsDetailed.loc[rsDetailed['Season'] == 2016]
Data2016 = pd.DataFrame(columns=['Season','Team','AvgPointSpread','Wins',
                                                      'Losses','FieldGoalAvg','ThreePointAvg',
                                                      'FreeThrowAvg','ReboundAvg','AssistAvg',
                                                      'TurnOverAvg','StealAvg','BlockAvg',
                                                      'PersonalFoulAvg'],index=Teams16.as_matrix())
#print(rsData03)
#print(Data2003)

for team in Teams16:
    wteamdata = rsData16.loc[rsData16['Wteam'] == team]
    lteamdata = rsData16.loc[rsData16['Lteam'] == team]
    #teamdata = rsData03.loc[(rsData03['Wteam'] == team) | (rsData03['Lteam'] == team)]
    
    tGames = len(wteamdata.index) + len(lteamdata.index)
    tWins = len(wteamdata.index)
    tLosses = len(lteamdata.index)
    tfgm = wteamdata['Wfgm'].sum() + lteamdata['Lfgm'].sum()
    tfga = wteamdata['Wfga'].sum() + lteamdata['Lfga'].sum()
    tfgm3 = wteamdata['Wfgm3'].sum() + lteamdata['Lfgm3'].sum()
    tfga3 = wteamdata['Wfga3'].sum() + lteamdata['Lfga3'].sum()
    tftm = wteamdata['Wftm'].sum() + lteamdata['Lftm'].sum()
    tfta = wteamdata['Wfta'].sum() + lteamdata['Lfta'].sum()
    tor = wteamdata['Wor'].sum() + lteamdata['Lor'].sum()
    tdr = wteamdata['Wdr'].sum() + lteamdata['Ldr'].sum()
    tast = wteamdata['Wast'].sum() + lteamdata['Last'].sum()
    tto = wteamdata['Wto'].sum() + lteamdata['Lto'].sum()
    tstl = wteamdata['Wstl'].sum() + lteamdata['Lstl'].sum()
    tblk = wteamdata['Wblk'].sum() + lteamdata['Lblk'].sum()
    tpf = wteamdata['Wpf'].sum() + lteamdata['Lpf'].sum()
    tScore = wteamdata['Wscore'].sum() + lteamdata['Lscore'].sum()
    tAgainst = wteamdata['Lscore'].sum() + lteamdata['Wscore'].sum()
    
    aps = (tScore - tAgainst)/tGames
    fga = tfgm/tfga
    fga3 = tfgm3/tfga3
    fta = tftm/tfta
    reb = (tor+tdr)/tGames
    ast = tast/tGames
    tover = tto/tGames
    stl = tstl/tGames
    blk = tblk/tGames
    foul = tpf/tGames
    
    Data2016.loc[team] = pd.Series({'Season':2016,'Team':team,'AvgPointSpread':aps,'Wins':tWins,
                   'Losses':tLosses,'FieldGoalAvg':fga,'ThreePointAvg':fga3,
                   'FreeThrowAvg':fta,'ReboundAvg':reb,'AssistAvg':ast,
                   'TurnOverAvg':tover,'StealAvg':stl,'BlockAvg':blk,
                   'PersonalFoulAvg':foul})
    
    
Data2016.head()
Data2016.to_csv('2016.csv')

In [8]:
rsDetailed.head()

,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot,Wfgm,Wfga,...,Lfga3,Lftm,Lfta,Lor,Ldr,Last,Lto,Lstl,Lblk,Lpf
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14


In [9]:
Data2016.head()

,Season,Team,AvgPointSpread,Wins,Losses,FieldGoalAvg,ThreePointAvg,FreeThrowAvg,ReboundAvg,AssistAvg,TurnOverAvg,StealAvg,BlockAvg,PersonalFoulAvg
1314,2016,1314,12.7059,28,6,0.478738,0.313894,0.737589,40.9706,18.1176,10.9118,6.88235,4.38235,17.9118
1462,2016,1462,10.3125,27,5,0.452012,0.361607,0.730769,41,16.375,12.75,7.3125,3.0625,20.1875
1452,2016,1452,12.5882,26,8,0.452195,0.328549,0.669841,39.1765,14.7647,14,9.94118,2.52941,23.3824
1246,2016,1246,11.4118,26,8,0.478561,0.370487,0.683398,38.5882,14.4118,11,5.61765,5.64706,20.4706
1231,2016,1231,13.375,25,7,0.501073,0.414698,0.723027,37.375,15.9688,13.6562,6.9375,3.96875,18.1875
